In [4]:
# main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/
# how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k video
from bs4 import BeautifulSoup 
import requests 
import pandas as pd

### Read Postcode Data from Wikipeadia


In [6]:
# read data

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

source = requests.get(url).text
soup = BeautifulSoup(source,'lxml')

table = soup.find('table')
#table.prettify()

### Create Dataframe

In [7]:
# parse data and create dataframe 
postcode =[]
borough =[]
neighbourhood =[]

for tr in table.find_all('tr'):
    i = 1 
    for td in tr.find_all('td'):
        if i == 1:
            postcode.append(td.text)    
        elif i == 2:    
            borough.append(td.text)
        elif i == 3:        
            neighbourhood.append(td.text.replace('\n',''))
        i +=1
        
dict = {'Postcode': postcode, 'Borough': borough, 'Neighbourhood': neighbourhood}        

df = pd.DataFrame.from_dict(dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Remove Borough = 'Not assigned'

In [34]:
# delete  rows with Borough = 'Not assigned'
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Generate Groupby Postcode Result DataFrame

In [111]:
# group by Postcode and generate sets for Borough,Neighbourhood 
boroughLst = df.groupby(['Postcode'])['Borough'].apply(set)
neighbourhoodLst = df.groupby(['Postcode'])['Neighbourhood'].apply(set)

# create df_result by mergin 2 sets on join Postcode
df_result = pd.merge(pd.DataFrame(boroughLst), pd.DataFrame(neighbourhoodLst), how='outer', on=['Postcode'])

# reset index to make Postcode columns 
df_result.reset_index(inplace=True)

# convert sets to String 
for i in range(len(result)-1):
    df_result.Neighbourhood[i] = ', '.join(df_result.Neighbourhood[i])
    df_result.Borough[i] = ', '.join(df_result.Borough[i])

# Update    Neighbourhood = 'Not assigned' with  Borough value. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
def myFun(Neighbourhood, Borough):
    if Neighbourhood =='Not assigned':
        return Borough
    else:
        return Neighbourhood
df_result.Neighbourhood = df_result.apply(lambda row :  myFun(row.Neighbourhood, row.Borough) , axis = 1) 

# show head 
df_result.head()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Morningside, Guildwood, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [112]:
# M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma 
df_result[df_result.Postcode =='M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [114]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.
df_result[df_result.Postcode =='M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


### Print Shape Result

In [115]:
df_result.shape

(103, 3)